In [117]:
import pandas as pd
import numpy as np
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat

# Авторизация в сервисе GigaChat
chat = GigaChat(credentials='ODYzNzU0NWQtMDc2MC00NDljLTk2YzQtNzhhZjEzY2IwYmY0Ojk4YmM1OGJkLTZlMGItNGFmNi1hNGZlLTU5ZWUxNGJiNGQzNA==', verify_ssl_certs=False, temperature=0.7)

messages = [
    SystemMessage(
        content="Ты аналитик в компании РЖД, необходмо проанализировать данные респондента. Ты даёшь свой ответ только в виде % ухода пользователя и обоснование"
    )
]

In [118]:
def parse_xls(filename: str):
    file = pd.read_excel(filename)
    rows = file.loc
    lst = []
    for row in file.itertuples(index=False):
        rw = []
        for r in row:
            rw.append(r)
        lst.append(rw)
    return lst

In [119]:
waitings = parse_xls("Интересы.xls")
zayavki = parse_xls("Обращения.xls")
obama = parse_xls("Объёмы перевозок.xls")
all_data = [waitings, zayavki, obama]

In [120]:
id = int(input("Введите ID:"))
for i in all_data:
    for k in i:
        for z in k:
            if z == id:
                print(k)
                break


['01.09.2023 13:19', 'Холодный звонок', '1.Продажа  ЦФТО', 'Завершен неудачно', 12200, 'ЦПУ МУРОМ', nan, nan, nan, nan, '10. Холодный звонок', '0001-233528', 'Интерес "Холодный звонок" от 01.09.2023']
['16.03.2023 09:03', 'Холодный звонок', '1.Продажа  ЦФТО', 'Завершен неудачно', 12200, 'ЦПУ МУРОМ', nan, nan, nan, nan, '10. Холодный звонок', '0001-057104', 'Интерес "Холодный звонок" от 16.03.2023']
['22.04.2022 14:24', 'Привлечение клиента', '1.Продажа  ЦФТО', 'Завершен неудачно', 12200, 'ЦПУ МУРОМ', nan, nan, nan, nan, '10. Холодный звонок', '0000-041922', 'Интерес "Привлечение клиента" от 22.04.2022']
['26.03.2024 09:10', 'Холодный звонок', '1.Продажа  ЦФТО', 'Завершен неудачно', 12200, 'ЦПУ МУРОМ', nan, nan, '04.10.2024 10:00:00', 'Повторный звонок', '10. Холодный звонок', '0001-079512', 'Интерес "Холодный звонок" от 26.03.2024']


In [121]:
# Вводим ID Получаем  -> list ["ID", "%", "Вывод", "Рекомендации"]

class ML_AGEENT:
    def __init__(self) -> None:
        self.dataframe_interesting = pd.read_excel("Интересы.xls")
        self.dataframe_waitings = pd.read_excel("Обращения.xls")
        self.dataframe_transporting = pd.read_excel("Сортированный_Объем Перевозок.xlsx")
        self.dataframe_Vladimyr = pd.read_excel("МС_Владимирская область.xls")
        self.dataframe_Kirov = pd.read_excel("МС_Кировская область.xls")
        self.dataframe_NNov = pd.read_excel("МС_Нижегородская область.xls")
        self.dataframe_MaryEl = pd.read_excel("МС_Республика Марий Эл.xls")
        self.dataframe_Mordoviya = pd.read_excel("МС_Республика Мордовия.xls")
        self.dataframe_Tatarstan = pd.read_excel("МС_Республика Татарстан.xls")
        self.dataframe_Ydmurt = pd.read_excel("МС_Республика Удмуртия.xls")
        self.dataframe_Chyvash = pd.read_excel("МС_Республика Чувашия.xls")
        self.oblast_list = [
                            self.dataframe_Vladimyr, 
                            self.dataframe_Kirov, 
                            self.dataframe_NNov, 
                            self.dataframe_MaryEl, 
                            self.dataframe_Mordoviya, 
                            self.dataframe_Tatarstan,
                            self.dataframe_Ydmurt,
                            self.dataframe_Chyvash
                            ]


    def research(self, id:str) -> list:
        
        _, personal_data = self.research_oblasts(id)
        _, personal_seasons = self.seasons(id)
        _, personal_dop_info, sum = self.research_directions(id)
        persnal_problems = self.research_obrash(id)
        persnal_inters = self.research_interesting(id)
        
        prompt = f"""У меня есть информация о клиенте железнодорожной компании, включая ключевое число:

У нас есть число: {sum}. Если данное число равно '0', мы выставляем данному клиенту 100% вероятность ухода. В этом случае необходимо предоставить способы возвращения данного клиента.
Данные клиента, связанные с размером его юридического лица, платежеспособностью и общими рисками данного бизнеса: {personal_data}.
Данные о его грузоперевозках за последние два года: {personal_seasons}.
Данные, которые связаны с общими способами взаимодействия компании с клиентом (обращения на консультирование, поддержку, жалобы, благодарности и т.д.): {persnal_inters}.
Данные, содержащие записи об возникших проблемах клиента: {persnal_problems}.
На основе предоставленных данных, пожалуйста, проанализируйте вероятность ухода клиента и предложите рекомендации для его возврата, если вероятность составляет больше 45%."""
        messages.append(HumanMessage(content=prompt))
        res = chat(messages)
        messages.append(res)
        return res.content
        ...
    
    def research_oblasts(self, id:str) -> str:
        for i in self.oblast_list:
            filtred = i[i["ID"] == id][["ID",
                                        "Размер компании.Наименование", 
                                        "Карточка клиента (внешний источник).Индекс платежной дисциплины Описание", 
                                        "Карточка клиента (внешний источник).Индекс финансового риска Описание", 
                                        "Госконтракты.Тип контракта"]]
            if len(filtred.values.tolist()) >=1:
                break
            else:
                pass
        
        all_data = filtred.values.T.tolist()
        # print(all_data[0], all_data[1])
        all_data = [list(set(sublist)) for sublist in all_data]
        # print(all_data[0], all_data[1])
        
        final = "Этот клиент - " +   str(all_data[1]) + " C " + str(all_data[2]) + " С индексом финального риска: " + str(all_data[3]) + " И в гос контрактах он является: " + str(all_data[4])
        # print(final)
        return filtred, final
    
    def seasons(self, id:str) ->  str:
        
        filtred = self.dataframe_transporting[self.dataframe_transporting["ID"] == id]
        years = filtred.columns.values.tolist()[5:]
        month = filtred[years].values.T.tolist()
        month_normal = years[0::2]
        month_in_numbers =  [[sum(subarray)] for subarray in month]
        month_in_tons = month_in_numbers[1::2]
        month_in_money = month_in_numbers[0::2]
        text = ""
        for i in range(len(month_normal)):
            text += f"Клиент в год, месяц - {month_normal[i]} перевёз {month_in_tons[i]} тон груза  на сумму: {month_in_money[i]}\n"
            
        # print(text)
        
        return filtred, text

    def research_directions(self, id:str):
        
        filtred=self.dataframe_transporting[self.dataframe_transporting['ID']==id]
        all_data2 = filtred.values.T.tolist()
        all_data1 = [x for x in all_data2 if x != []]
        empty_list = []
        text=""
        sum=0
        len_data = len(all_data1)
        print(len_data)
        print(all_data1)
        try:
            for i in range(len(all_data[3])):

                text+="| Этот клиент везет из - " + all_data[1][i] + " в - " + all_data[2][i] + "| Код товара - " + str(all_data[3][i])+"| Категория груза - "+all_data[4][i]+"|\n"
                
            print (text)
        except Exception:
            text += "Грузоперевозок не было найдено"
        
        
        years = filtred.columns.values.tolist()[5:]
        month = filtred[years].values.T.tolist()
        
        for i in month[-9:]:
            
            for j in i:
                
                sum+=j
                
        
        
        # print(years,month)
        
        
        return all_data, text,sum

    def research_interesting(self, id:str) -> str:
        filtred = self.dataframe_interesting[self.dataframe_interesting["ID"] == id][["Состояние", "Канал первичного интереса"]]

        all_data = filtred.values.T.tolist()
        all_data = [list(set(sublist)) for sublist in all_data]

        string = ""
        ls = []

        for i in filtred.values:
            string += str(i[0]) + " " + str(i[1]) + ", "

        for i in all_data[0]:
            for j in all_data[1]:
                if string.count(f"{i} {j}") != 0:
                    ls.append([str(i) + " " + str(j), string.count(f"{i} {j}")])

        for i in ls:
            string += i[0] + " " + str(i[1]) + "\n"

        return string
    

    def research_obrash(self, id:str) -> str:
        filtred = self.dataframe_waitings[self.dataframe_waitings["ID"] == id][["Тема вопроса"]]

        all_data = filtred.values.T.tolist()
        try:
            
            string = ""
            for i in all_data:
                string += i[0] + "\n"

            # print(string)
        except:
            string = "Обращений не обнаружено"
        
        return string

In [122]:
agent = ML_AGEENT()

In [123]:
agent.research(id) 


0
[]


C:\Users\Admin\AppData\Local\Temp\ipykernel_27216\2062225374.py:45: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = chat(messages)


"На основе предоставленных данных, вероятность ухода данного клиента составляет 100%, так как ключевое число '0' указывает на отсутствие каких-либо перевозок со стороны клиента. Это приводит к отсутствию доходов от этого клиента для компании.\n\nРекомендации по возвращению данного клиента могут включать следующие шаги:\n\n1. Проведение анализа рынка и конкурентов для определения возможных причин отказа от услуг компании.\n2. Разработка персонализированных предложений и скидок для возобновления интереса клиента.\n3. Организация встреч и переговоров с целью обсуждения возможностей сотрудничества и поиска решений проблем, которые могли привести к уходу клиента.\n4. Использование программ лояльности и бонусов для стимулирования повторного обращения клиента.\n5. Оценка качества предоставляемых услуг и возможная оптимизация процессов для улучшения клиентского опыта.\n\nВажно также учитывать, что возвращение клиента может потребовать значительных усилий и ресурсов, поэтому решение о возвращен